In [1]:
import pandas as pd
import time
import gspread
from google.oauth2.service_account import Credentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from math import floor,ceil
import pmdarima as pm
from pmdarima.model_selection import train_test_split as pm_train_test_split
#from prophet import Prophet
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
import itertools
import pandas_market_calendars as mcal
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib as mpl
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import plotly.graph_objs as go
from pylab import rcParams
import warnings
warnings.simplefilter("ignore")

In [2]:
scope = ['https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive']

creds = Credentials.from_service_account_file("/work/cuenta de servicio.json", scopes=scope)
client = gspread.authorize(creds)

worksheet_data = client.open("SP_500_data_clean").get_worksheet(0).get_all_records()
worksheet = client.open("SP_500_data_clean").get_worksheet(0)

In [3]:
# El ticker BRK.B no se importa debido a su ausencía de datos y a su pertenencia al indice NYSE
df = get_as_dataframe(worksheet)

df.set_index("Date", inplace=True)
df.index = pd.to_datetime(df.index)
df = df.asfreq('D')
df.dropna(how="all", axis=0, inplace=True)
df = df[["AAPL","MSFT","AMZN","TSLA","GOOGL","GOOG","NVDA","META","UNH"]]

In [4]:
def is_stationary(data, symbol):

    ajuste = ExponentialSmoothing(data[symbol]).fit()
    valores_suavizados = ajuste.fittedvalues  
    result = adfuller(valores_suavizados)

    return  result[1] < 0.05 

In [5]:
def estacionalizar(data,symbol):
    
  if not is_stationary(data, symbol):
    print(f"La serie {symbol} no es estacionaria")
    d = 1
    #diff_data = np.diff(data[symbol], n=d)
    diff_data = data[[symbol]].diff(d).dropna()
    #diff_data = pd.DataFrame(diff_data, columns=[symbol])
    
    while not is_stationary(diff_data, symbol):
        d += 1
        #diff_data = np.diff(diff_data[symbol], n=1)
        diff_data = data[[symbol]].diff(d).dropna()
        #diff_data = pd.DataFrame(diff_data, columns=[symbol])
    return diff_data[[symbol]], d
  
  else:

    print(f"la serie {symbol} es estacionaria, no se requieren transformaciones")
    d = 0
    return data[[symbol]],d

In [6]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def pred_arima(data, symbol, ratio):

  obs = ceil(len(data[symbol])*ratio)

  train, test = data[:obs], df[obs:] #pm_train_test_split(estacionalizar(df,"AAPL")[0], train_size=obs)
  train_est, d = estacionalizar(train, symbol)

  model = pm.auto_arima(train_est[symbol], seasonal=False)
  preds, conf_int = model.predict(n_periods = test.shape[0], return_conf_int=True)

  
  x = pd.DataFrame(preds.values,columns=[symbol], index=test[symbol].shift(d).index)
  y_pred = x[[symbol]] + data[[symbol]][obs-d:]
  y_pred = y_pred[[symbol]][d:]
  mse = mean_squared_error(test[symbol].values, y_pred[symbol].values)
  rmse = np.sqrt(mse)
  mae = mean_absolute_error(test[symbol].values, y_pred[symbol].values)
  mape = mean_absolute_percentage_error(test[symbol].values, y_pred[symbol].values)
  metr = {}

  metr[f"{symbol}"] = {"mse":mse,"rmse":rmse, "mae":mae,"mape":mape}
  df_metr = pd.DataFrame(metr).T
  df_metr["order"] = str(model.order) 
  df_metr["diff"] = d
  df_metr["summary"] = model.summary()

#   plt.plot(train[symbol], label= "Entrenamiento")
#   plt.plot(test[symbol], label = "Prueba")
#   plt.plot(x[symbol] + data[symbol][obs-d:], label = "Prediccción") #x["AAPL"] + test["AAPL"].shift(3)
#   plt.title(f"Resultados predcción modelo arima orden {model.order} para el activo {symbol}")
#   plt.legend()
#   plt.show()

  return df_metr

In [7]:
df_list_ar = []
for symbol in df.columns:
    metricas = pred_arima(df,symbol, 0.8)
    df_list_ar.append(metricas)


La serie AAPL no es estacionaria
La serie MSFT no es estacionaria
La serie AMZN no es estacionaria
La serie TSLA no es estacionaria
La serie GOOGL no es estacionaria
La serie GOOG no es estacionaria
La serie NVDA no es estacionaria
La serie META no es estacionaria
La serie UNH no es estacionaria


In [8]:
df_metricas = pd.concat(df_list_ar)
df_metricas

,mae,mape,mse,rmse,order,diff,summary
AAPL,0.377311,0.249487,0.363319,0.602760,"(5, 0, 1)",3,SARIMAX Results...
MSFT,0.606226,0.232455,0.990942,0.995461,"(5, 0, 1)",3,SARIMAX Results...
AMZN,0.090887,0.066181,0.656438,0.810209,"(5, 0, 1)",3,SARIMAX Results...
TSLA,0.879694,0.431554,0.831476,0.911853,"(0, 0, 2)",3,SARIMAX Results...
GOOGL,0.084276,0.070494,0.419983,0.648061,"(5, 0, 1)",3,SARIMAX Results...
GOOG,0.089081,0.074236,0.458530,0.677149,"(5, 0, 1)",3,SARIMAX Results...
NVDA,0.100656,0.052909,0.458365,0.677026,"(5, 0, 0)",2,SARIMAX Results...
META,0.073689,0.039511,0.913344,0.955690,"(3, 0, 2)",3,SARIMAX Results...
UNH,0.638404,0.126019,1.239577,1.113363,"(5, 0, 0)",2,SARIMAX Results...


In [9]:
worksheet_met = client.open("SP_500_metricas").get_worksheet(0)
worksheet_met.clear()
set_with_dataframe(worksheet=worksheet_met,dataframe=df_metricas,resize=True, include_index=True)

In [10]:
def forecast_arima(data, symbol, ratio=0,steps=30):

  data_sub = data[[symbol]][ceil(len(data[symbol])*ratio):]
  data_est, d = estacionalizar(data_sub, symbol)

  model = pm.auto_arima(data_est[symbol], seasonal=False)
  preds, conf_int = model.predict(n_periods = steps, return_conf_int=True)

  # crear un calendario con los días en los que la bolsa de Nueva York está abierta
  last_date = data.tail(1).index + timedelta(1)
  l_year = last_date.year[0] 
  l_month = last_date.month[0]
  l_day = last_date.day[0]
  future_date = last_date + timedelta(steps*3)
  f_year = future_date.year[0] 
  f_month = future_date.month[0]
  f_day = future_date.day[0]
  nyse = mcal.get_calendar('NYSE')
  schedule = nyse.schedule(start_date=f'{l_year}-{l_month}-{l_day}', end_date=f'{f_year}-{f_month}-{f_day}')

  next_dates = schedule.index.values[:steps]

  y_pred = data[symbol].values.tolist()[-d:]
  max_len = max(len(y_pred),len(preds))
  for i in range(max_len):
    y_pred.append(y_pred[i] + preds.tolist()[i])

  

  forecast_df = pd.DataFrame(y_pred[d:], columns=[f"{symbol}"],index=next_dates)
  
  

  # plt.plot(data[symbol], label= "Datos")
  # plt.plot(forecast_df[f"{symbol}_pred"], label = "Prónostico")
 
  
  # plt.title(f"Resultados predcción a {steps} días modelo arima orden {model.order} para el activo {symbol}")
  # plt.legend()
  # plt.show()
  return forecast_df

In [11]:
df_fore_ar = []
for symbol in df.columns:
    preds = forecast_arima(df,symbol,steps=30)
    df_fore_ar.append(preds)

La serie AAPL no es estacionaria
La serie MSFT no es estacionaria
La serie AMZN no es estacionaria
La serie TSLA no es estacionaria
La serie GOOGL no es estacionaria
La serie GOOG no es estacionaria
La serie NVDA no es estacionaria
La serie META no es estacionaria
La serie UNH no es estacionaria


In [12]:
df_forecast = pd.concat(df_fore_ar,axis=1)
df_forecast

,AAPL,MSFT,AMZN,TSLA,GOOGL,GOOG,NVDA,META,UNH
2023-05-23,174.666437,321.750652,114.561384,188.508984,123.690706,124.481261,311.826221,250.174949,482.336530
2023-05-24,174.565719,320.446407,115.237442,189.001333,123.664209,124.359166,314.332468,246.794041,480.533076
2023-05-25,174.161151,322.310008,114.815291,188.773294,124.112777,124.930166,314.327916,247.676396,482.146224
2023-05-26,175.591169,323.082926,115.547409,190.640067,123.550466,124.362003,318.776259,249.138558,480.794345
2023-05-30,175.558418,322.433467,115.080817,189.195616,122.861932,123.484247,316.435191,245.757909,481.769925
2023-05-31,174.851798,323.679660,115.371736,191.188454,124.028814,124.855917,317.588086,247.036758,480.619063
2023-06-01,175.739452,324.568813,114.860739,190.123676,123.039178,123.816848,316.119849,248.611960,481.953227
2023-06-02,175.591347,322.879995,115.376447,191.346554,122.843501,123.477868,318.261475,245.431368,480.653911
2023-06-05,175.021706,324.493758,114.890555,190.029497,123.783621,124.580345,316.650541,246.785017,481.929332
2023-06-06,176.146976,324.972371,115.383342,191.528094,123.155054,123.937076,319.039309,248.563896,480.684628


In [13]:
worksheet_forecast = client.open("SP_500_forecast").get_worksheet(0)
worksheet_forecast.clear()
set_with_dataframe(worksheet=worksheet_forecast,dataframe=df_forecast,resize=True, include_index=True)

# Random forest

In [14]:
# Manipulación datos
import numpy as np
import pandas as pd

from numpy import array
#from keras.models import Sequential
#from keras.layers import Dense
from numpy import asarray
from pandas import DataFrame
from pandas import concat

# Gráficos
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from matplotlib import pyplot
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
%matplotlib inline

# Modelación y pronóstico
#Random Forest:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
##Regressor:
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregMultiOutput import ForecasterAutoregMultiOutput
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from sklearn.metrics import mean_squared_error


from joblib import dump, load
from copy import copy


from datetime import datetime
from math import sqrt
# Configuración de warnings:
import warnings
# warnings.filterwarnings('ignore')

In [15]:
def forecast_randomForest(data,symbol,ratio=0.8, steps=30):

    #Especificamos los últimos datos que servirán para la parte de testing o capacidad predictiva del modelo.
    steps_fut = steps
    #Aplicamos dicha proporción a la definición de la parte de entrenamiento y de prueba.

    obs = ceil(len(data[symbol])*ratio)

    data_train, data_test = data[:obs], data[obs:]

    # Cálculo de hiperparámetros por grid search
    # ==============================================================================
    steps_test = len(data_test)
    forecaster = ForecasterAutoreg(
                    regressor = RandomForestRegressor(random_state=123, n_jobs=-1),
                    lags      = 10 # Este valor se reemplazará en la grilla
                    
                )

    # Lags used as predictors
    lags_grid = [5,10,15]

    # Regressor's hyperparameters
    param_grid = {'n_estimators': [100,500],
                'max_depth': [10,15,20],
                #'min_samples_split': [2, 5, 10],
                #'min_samples_leaf': [1, 2, 4],
                # 'max_features': ['auto', 'sqrt', 'log2', None],
                #'bootstrap':[True,False],
                # 'min_impurity_decrease': [0.0, 0.1, 0.2],
                # 'max_leaf_nodes': [None, 10, 20]
                }

    results_grid = grid_search_forecaster(
                            forecaster         = forecaster,
                            y                  = data_train[symbol],
                            param_grid         = param_grid,
                            lags_grid          = lags_grid,
                            steps              = steps,
                            refit              = True,
                            metric             = 'mean_squared_error',
                            initial_train_size = int(len(data_train[symbol])*0.5),
                            fixed_train_size   = False,
                            return_best        = True,
                            verbose            = False
                               

                )

    lags = len(results_grid["lags"][0])
    #boots = results_grid["bootstrap"].iloc[0]
    md = results_grid["max_depth"].iloc[0]
    n_e = results_grid["n_estimators"].iloc[0]

    
    ### Aquí debe ir tu codigo, 
    regressor = RandomForestRegressor(max_depth= md, n_estimators= n_e, random_state=123,n_jobs=-1)
    ### Hasta aquí modificas

    forecaster_test = ForecasterAutoreg(
                    regressor = regressor,
                    lags      = lags
                )

    forecaster_test.fit(y=data_train[symbol])

    predictions_test = forecaster_test.predict(steps=steps_test)

    mse = mean_squared_error(data_test[symbol].values, predictions_test)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(data_test[symbol].values, predictions_test)

    def mean_absolute_percentage_error(y_true, y_pred):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    mape = mean_absolute_percentage_error(data_test[symbol].values, predictions_test)
    metr = {}

    metr[f"Random Forest {symbol}"] = {"mse":mse,"rmse":rmse, "mae":mae,"mape":mape}
    df_metr = pd.DataFrame(metr).T
    df_metr["depth"] = md
    df_metr["estimators"] = n_e
    df_metr["lags"] = lags

    forecaster_fut = ForecasterAutoreg(
                    regressor = regressor,
                    lags      = lags
                )

    forecaster_fut.fit(y=data[symbol])

    importance = forecaster_fut.get_feature_importance()
    importance["Symbol"] = symbol

    predictions_fut = forecaster_fut.predict(steps=steps_fut)

    # crear un calendario con los días en los que la bolsa de Nueva York está abierta
    last_date = data.tail(1).index + timedelta(1)
    l_year = last_date.year[0] 
    l_month = last_date.month[0]
    l_day = last_date.day[0]
    future_date = last_date + timedelta(steps_fut*3)
    f_year = future_date.year[0] 
    f_month = future_date.month[0]
    f_day = future_date.day[0]
    nyse = mcal.get_calendar('NYSE')
    schedule = nyse.schedule(start_date=f'{l_year}-{l_month}-{l_day}', end_date=f'{f_year}-{f_month}-{f_day}')

    next_dates = schedule.index.values[:steps_fut]
    forecast_rf = pd.DataFrame(predictions_fut.values, columns=[f"{symbol}"],index=next_dates)

    return forecast_rf, df_metr, importance

In [16]:
df_for_list = []
df_metr_list = []
df_import_list = []
for symbol in df.columns:
    forecast_rf, metr_rf, importance = forecast_randomForest(df,symbol)
    df_for_list.append(forecast_rf)
    df_metr_list.append(metr_rf)
    df_import_list.append(importance)

Number of models compared: 18.
loop param_grid:   0%|                                              | 0/6 [00:00<?, ?it/s]

In [17]:
df_forecast_random_forest = pd.concat(df_for_list, axis=1)
df_metricas_random_forest = pd.concat(df_metr_list)
df_importance_random_forest = pd.concat(df_import_list)

In [18]:
worksheet_forecast_rf = client.open("SP_500_forecast_RF").get_worksheet(0)
worksheet_forecast_rf.clear()
set_with_dataframe(worksheet=worksheet_forecast_rf,dataframe=df_forecast_random_forest,resize=True, include_index=True)


In [19]:
worksheet_metricas_rf = client.open("Hoja de SP_500_metricas_RF").get_worksheet(0)
worksheet_metricas_rf.clear()
set_with_dataframe(worksheet=worksheet_metricas_rf,dataframe=df_metricas_random_forest,resize=True, include_index=True)

In [20]:
worksheet_importance_rf = client.open("SP_500_Importance_RF").get_worksheet(0)
worksheet_importance_rf.clear()
set_with_dataframe(worksheet=worksheet_importance_rf,dataframe=df_importance_random_forest,resize=True, include_index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ada87ab4-a3d4-4fc2-b63c-627801a13813' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>